In [1]:
import os
import datajoint as dj
import pathlib

In [3]:
from ephys import reference, acquisition, tracking, behavior, ephys, analysis
mlims = dj.create_virtual_module('mlims', dj.config['custom'].get('mlims.database'))

Connecting thinh@datajoint.it.ntnu.no:3306


# The Ephys Pipeline